
# QUBO for Facility Location (Plant Opening + Assignment) — Colab-Ready

This notebook shows how to translate an **Uncapacitated Facility Location** problem into a **QUBO** and solve it with a **simulated annealer** (`neal`).

**Model (uncapacitated):**
- Binary \(y_i\): 1 if plant \(i\) is open
- Binary \(z_{ij}\): 1 if customer \(j\) is assigned to plant \(i\)

**Objective:** minimize fixed opening + shipping cost  
**Constraints:**
1) Each customer assigned exactly once: \(\sum_i z_{ij}=1\)  
2) Assign only to open plants: \(z_{ij} \le y_i\)

We convert constraints into quadratic **penalties** and build a single QUBO energy \(E(x) = x^T Q x\). Then we solve with a simulated annealer.

> Penalty weights \(A, B\) must be **large enough** to dominate cost tradeoffs. We set them based on data scale and you can tune them if needed.


In [ ]:

# If running in Colab, install the required packages.
# (In many environments, these may already be available.)
try:
    import dimod, neal  # noqa: F401
    HAVE_DEPS = True
except Exception:
    HAVE_DEPS = False

if not HAVE_DEPS:
    %pip -q install dimod neal


In [ ]:

import numpy as np, pandas as pd

rng = np.random.default_rng(8)

# Small instance: 4 plants, 7 customers
P = [f"P{i+1}" for i in range(4)]
C = [f"C{j+1}" for j in range(7)]

# Fixed open costs for plants
f = pd.Series(rng.integers(2000, 6001, size=len(P)), index=P, name="fixed_cost")

# Per-unit ship cost from plant i to customer j (can include production cost)
c = pd.DataFrame(rng.integers(5, 41, size=(len(P), len(C))), index=P, columns=C)

# Demand by customer (unit = 1 customer; shipping cost is multiplied by demand)
# Here we keep demand = 1 to match pure assignment; scaling works too.
d = pd.Series(np.ones(len(C), dtype=int), index=C, name="demand")

# Penalties: scale by the max shipping cost to be safe
M = (c.values * d.values).max()
A = 10 * M  # assignment = 1
B = 10 * M  # open-if-assigned

print("Plants:", list(P))
print("Customers:", list(C))
print("\nFixed costs:\n", f.to_string())
print("\nShipping cost matrix (c_ij):\n", c.to_string())
print("\nPenalty weights: A =", A, " B =", B)


In [ ]:

from collections import defaultdict

# Index helper: map (var_type, i[, j]) -> label string
def y_label(i):
    return f"y|{i}"

def z_label(i,j):
    return f"z|{i}|{j}"

# Build Q matrix as a dict-of-dict (dimod format can accept flat dict too)
Q = defaultdict(float)

# Linear terms
for i in P:
    Q[(y_label(i), y_label(i))] += float(f[i])

for i in P:
    for j in C:
        Q[(z_label(i,j), z_label(i,j))] += float(c.loc[i,j]*d[j] - A + B)

# Quadratic: assignment-equals-1 coupling among z for the same customer
for j in C:
    for idx_i, i in enumerate(P):
        for k in P[idx_i+1:]:
            Q[(z_label(i,j), z_label(k,j))] += float(2*A)

# Quadratic: open-if-assigned coupling between z_ij and y_i
for i in P:
    for j in C:
        Q[(z_label(i,j), y_label(i))] += float(-B)

# Convert to dimod BinaryQuadraticModel
import dimod

# Flatten to {(u,v): coeff} dict as required by from_qubo
Q_flat = dict(Q)

bqm = dimod.BinaryQuadraticModel.from_qubo(Q_flat)

print("Number of variables in QUBO:", len(bqm.variables))
print("Number of QUBO interactions:", len(bqm.quadratic))


In [ ]:

import neal

sampler = neal.SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm, num_reads=2000)
best = sampleset.first  # lowest energy sample
print("Best energy:", best.energy)
print("Sample size:", len(sampleset))


In [ ]:

import pandas as pd

# Extract solution vectors
x = best.sample  # dict var->0/1

open_plants = [i for i in P if x[y_label(i)] == 1]
assign = {(i,j): x.get(z_label(i,j), 0) for i in P for j in C}

# Build assignment per customer
assign_by_customer = {j: [i for i in P if assign[(i,j)] == 1] for j in C}

# Compute costs
fixed_cost = sum(f[i] for i in open_plants)
ship_cost = 0.0
for j in C:
    for i in P:
        if assign[(i,j)] == 1:
            ship_cost += c.loc[i,j] * d[j]

total_cost = fixed_cost + ship_cost

# Validate constraints
violations = []
# each customer exactly one plant
for j in C:
    if sum(assign[(i,j)] for i in P) != 1:
        violations.append(f"Assignment count for {j} = {sum(assign[(i,j)] for i in P)} (should be 1)")

# z_ij <= y_i
for i in P:
    for j in C:
        if assign[(i,j)] == 1 and x[y_label(i)] != 1:
            violations.append(f"Open-if-assigned violated: z_{i}{j}=1 but y_{i}=0")

print("\nOPEN PLANTS:", open_plants)
print("Fixed cost:", int(fixed_cost))
print("Shipping cost:", int(ship_cost))
print("Total cost:", int(total_cost))

print("\nAssignments (customer -> plant):")
for j in C:
    plants_j = [i for i in P if assign[(i,j)] == 1]
    print(f"  {j}: {plants_j}")

if violations:
    print("\nVIOLATIONS:")
    for v in violations:
        print(" -", v)
else:
    print("\nAll constraints satisfied ✅")

# Tabular summary
df_assign = pd.DataFrame(
    [(i,j,assign[(i,j)]) for i in P for j in C if assign[(i,j)]==1],
    columns=["plant","customer","assigned"]
)
df_open = pd.DataFrame({"plant": P, "open": [1 if i in open_plants else 0 for i in P]})
display(df_open)
display(df_assign)



## (Optional) Exact MILP Comparison

For small instances, you can verify the QUBO solution quality by solving the **MILP** version with CBC via **PuLP**.


In [ ]:

# Comment out if you don't want the MILP comparison.
try:
    import pulp
    MILP_OK = True
except Exception:
    MILP_OK = False

if not MILP_OK:
    try:
        %pip -q install pulp
        import pulp
        MILP_OK = True
    except Exception:
        MILP_OK = False

if MILP_OK:
    prob = pulp.LpProblem("UFLP", pulp.LpMinimize)
    y = {i: pulp.LpVariable(f"y_{i}", lowBound=0, upBound=1, cat="Binary") for i in P}
    z = {(i,j): pulp.LpVariable(f"z_{i}_{j}", lowBound=0, upBound=1, cat="Binary") for i in P for j in C}

    prob += pulp.lpSum(f[i]*y[i] for i in P) + pulp.lpSum(c.loc[i,j]*z[(i,j)] for i in P for j in C)

    # Each customer assigned exactly once
    for j in C:
        prob += pulp.lpSum(z[(i,j)] for i in P) == 1

    # Open-if-assigned
    for i in P:
        for j in C:
            prob += z[(i,j)] <= y[i]

    prob.solve(pulp.PULP_CBC_CMD(msg=False))
    status = pulp.LpStatus[prob.status]
    obj = pulp.value(prob.objective)

    y_sol = {i: int(round(pulp.value(y[i]) or 0)) for i in P}
    z_sol = {(i,j): int(round(pulp.value(z[(i,j)]) or 0)) for i in P for j in C}

    print("MILP status:", status)
    print("MILP objective:", int(obj))
    print("MILP open plants:", [i for i in P if y_sol[i]==1])
    print("MILP assignments (customer->plant):")
    for j in C:
        print(" ", j, ":", [i for i in P if z_sol[(i,j)]==1])
else:
    print("PuLP not available and install failed; skipping MILP comparison.")
